In [46]:
import pandas as pd
import numpy as np

In [47]:
mkt_cap = pd.read_excel(r"Market Cap S5INFT 20190101-20231016.xlsx", parse_dates = ["DATES"])
stock_price = pd.read_excel(r"Stock Price S5INFT 20190101-20231016.xlsx", parse_dates = ["DATES"])

## Preprocessing on "mkt_cap" and "stock_price" (Daily data)

### Drop null values

In [48]:
def drop_null(df):
    df_1 = df.copy().fillna(0)
    l = []
    for x in df_1.index:
        if sum(df_1.iloc[x, 3:]) == 0:
            l.append(x)
    return df_1.drop(l).reset_index().drop('index',axis=1)

In [49]:
mc = drop_null(mkt_cap)
sp = drop_null(stock_price)

### Drop abnormal sample

In [50]:
def get_abnormal(df):
    ab = []
    for x in df.columns:
        if df[x].min() == 0:
            ab.append(x)
    return ab

In [51]:
for x in get_abnormal(mc):
    print (f"{x} has {list(mc.loc[:, x]).count(0)} null values in mkt_cap") 

PANW UW Equity has 709 null values in mkt_cap
STX UW Equity has 1 null values in mkt_cap
QCOM UW Equity has 1 null values in mkt_cap
ROP UW Equity has 1134 null values in mkt_cap


In [52]:
for x in get_abnormal(sp):
    print (f"{x} has {list(mc.loc[:, x]).count(0)} null values in stock price") 

PANW UW Equity has 709 null values in stock price
ROP UW Equity has 1134 null values in stock price


In [53]:
mc_1 = mc.drop(columns = ['PANW UW Equity', 'ROP UW Equity'])
sp_1 = sp.drop(columns = ['PANW UW Equity', 'ROP UW Equity'])

### Get Weekly Data

In [54]:
def get_weekly_data(df):
    df_1 = df.copy()
    for i in range(df_1.Week.max()):
        x = df_1.loc[df_1.Week == i+1]
        df_1 = df_1.drop(x.loc[x.Day < x.Day.max()].index)
    return df_1.drop("Day", axis=1).reset_index().drop('index',axis=1)

In [55]:
mc_2 = get_weekly_data(mc_1)
sp_2 = get_weekly_data(sp_1)

In [56]:
mc_3 = mc_2.drop(mc_2.loc[mc_2.Week == 251].index)
sp_3 = sp_2.drop(sp_2.loc[sp_2.Week == 251].index)

### Transfer Format

In [57]:
def transfer_format(df):
    y = pd.DataFrame(columns=['DATES', "Week", "Ticker", "x"])
    for i in range(len(df.columns)-2):
        x = pd.DataFrame(columns=['DATES', "Week", "Ticker", "x"])
        x['DATES'] = df.iloc[:, 0]
        x["Week"] = df.iloc[:, 1]
        x["Ticker"] = df.columns[i+2].split(" ")[0]
        x["x"] = df.iloc[:, i+2]
        y = pd.concat([y, x])
    return y.reset_index().drop('index',axis=1)
        

In [58]:
mc_4 = transfer_format(mc_3).rename(columns={"x": "mrk_cap"})
sp_4 = transfer_format(sp_3).rename(columns={"x": "price"})

### Save new datasets

In [59]:
mc_4.to_excel('mkt cap.xlsx',index=False)
sp_4.to_excel('stock price.xlsx',index=False)

# Merge

In [ ]:
jl = pd.read_excel(r"Job count.xlsx")

In [61]:
result = pd.merge(sp_4, mc_4)
x = pd.merge(result, jl, how='left', on = ["Ticker", "Week"])

In [68]:
merged= x.copy().fillna(0)
merged_1 = merged.drop(merged.loc[merged['count'] == 0].index).reset_index(drop=True)

In [94]:
merged_1 = merged_1.rename(columns={"count":"job count"})

### Get Lagged Variables

In [95]:
def lagged(df, n, variable, new):
    merged_1.loc[:, new] = 0
    for index, row in merged_1.iterrows():
        ticker = row["Ticker"]
        week = row["Week"]+n
        if week <= 250:
            merged_1.loc[index, new] = df.loc[(df["Ticker"] == ticker) & (df["Week"] == week)][variable].values[0]

In [97]:
lagged(sp_4, 13, "price", "sp_1q")
lagged(sp_4, 26, "price", "sp_2q")
lagged(sp_4, 39, "price", "sp_3q")
lagged(sp_4, 52, "price", "sp_4q")

/var/folders/78/mzvhkg9541qgk1th67dnww8m0000gn/T/ipykernel_32855/3378634190.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged_1.loc[:, new] = 0


In [98]:
lagged(mc_4, 13, "mrk_cap", "mc_1q")
lagged(mc_4, 26, "mrk_cap", "mc_2q")
lagged(mc_4, 39, "mrk_cap", "mc_3q")
lagged(mc_4, 52, "mrk_cap", "cm_4q")

/var/folders/78/mzvhkg9541qgk1th67dnww8m0000gn/T/ipykernel_32855/3378634190.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged_1.loc[:, new] = 0


In [99]:
merged_1.head()

,DATES,Week,Ticker,price,mrk_cap,Posted_year,Posted_week,job count,sp_1q,sp_2q,sp_3q,sp_4q,mc_1q,mc_2q,mc_3q,cm_4q
0,2019-01-04,1,CDW,77.88,1.168081e+10,2019.0,1.0,2215.0,100.79,113.79,122.40,142.15,1.482210e+10,1.661584e+10,1.771306e+10,2.042420e+10
1,2019-01-11,2,CDW,80.59,1.208727e+10,2019.0,2.0,2535.0,107.00,116.22,124.41,142.36,1.565434e+10,1.697068e+10,1.800394e+10,2.045438e+10
2,2019-01-18,3,CDW,81.69,1.225226e+10,2019.0,3.0,1804.0,105.39,112.51,123.91,136.85,1.541880e+10,1.642894e+10,1.793158e+10,1.966270e+10
3,2019-01-25,4,CDW,81.96,1.229275e+10,2019.0,4.0,2304.0,105.29,114.44,124.86,137.92,1.540417e+10,1.671076e+10,1.806906e+10,1.981644e+10
4,2019-02-01,5,CDW,83.55,1.253123e+10,2019.0,5.0,2426.0,109.09,116.34,131.20,130.45,1.592954e+10,1.683609e+10,1.885090e+10,1.874314e+10


### Add categories

In [81]:
add_data = pd.read_excel(r"categories.xlsx")

In [82]:
add_data.head()

,Ticker,category,Posted_year,Posted_week,count,company industry
0,CDW,Consultant,2021,42,466,Software & Computer Services
1,CDW,Consultant,2022,5,528,Software & Computer Services
2,CDW,Consultant,2019,7,7,Software & Computer Services
3,CDW,Consultant,2021,33,305,Software & Computer Services
4,CDW,Consultant,2022,32,650,Software & Computer Services


In [84]:
#industry
industry = add_data.loc[:, ["Ticker", "Posted_year", "Posted_week", "company industry"]]
industry = industry.drop_duplicates()
duplicated = industry[industry.iloc[:, 0:-1].duplicated()]
industry = industry.drop(duplicated.index)
x = pd.merge(merged_1, industry, how='left', on = ["Ticker", "Posted_year", "Posted_week"])

In [85]:
df_category = add_data.loc[:, ["Ticker", "Posted_year", "Posted_week"]]
df_category = df_category.drop_duplicates().reset_index().drop('index',axis=1)

In [ ]:
#categories
# However, it takes quite long time to run the code below.
for index, row in df_category.iterrows():
    ticker = row["Ticker"]
    year = row["Posted_year"]
    week = row["Posted_week"]
    for x in add_data["category"].unique():
        matching_rows = add_data.loc[(add_data.Ticker == ticker) & (add_data.Posted_year == year) & (add_data.Posted_week == week) & (add_data.category == x), "count"].values
        if len(matching_rows) > 0:
            df_category.at[index, x] = matching_rows[0]
        else:
            df_category.at[index, x] = 0

In [ ]:
matched = pd.merge(x, df_category, how='left', on = ["Ticker", "Posted_year", "Posted_week"])
matched.to_excel("Weekly Data.xlsx")